# Portfolio Optimization using Deep Reinforcement Learning
----

## 3.0 Stock Selection
---
Auto Encoders are employed to select the less volatile stocks by choosing stocks with less reconstruction error. These are the stocks that are included in the portfolio

### 3.1 Import Relevant Libraries

In [165]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [166]:
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import regularizers, optimizers

from sklearn import preprocessing

### 3.2 Load the Data

In [167]:
# Load the close prices dataset
prices_data = pd.read_csv('./datasets/close_prices.csv')

In [168]:
df = prices_data.copy()

In [169]:
df = df.reset_index(drop=True).set_index(['date'])

In [170]:
df.head()

,HCLTECH.NS,EICHERMOT.NS,HINDALCO.NS,INDUSINDBK.NS,GRASIM.NS,AXISBANK.NS,ONGC.NS,BRITANNIA.NS,BPCL.NS,RELIANCE.NS,...,POWERGRID.NS,TATAMOTORS.NS,UPL.NS,BAJAJFINSV.NS,ICICIBANK.NS,DIVISLAB.NS,TCS.NS,TECHM.NS,BAJFINANCE.NS,BHARTIARTL.NS
date,,,,,,,,,,,,,,,,,,,,,
2008-01-01,83.612503,41.000000,197.665359,131.000000,589.437805,196.690002,209.149994,149.000000,88.283333,662.662659,...,83.221893,146.884857,119.633331,2630.000000,225.454544,482.487488,269.250000,289.937500,43.721157,450.628632
2008-01-02,81.474998,41.000000,199.115448,132.199997,590.209290,208.990005,214.833328,151.960007,88.083336,659.233887,...,83.081268,152.729584,125.983330,2629.000000,236.309097,480.225006,265.250000,287.212494,45.858635,442.832764
2008-01-03,79.187500,46.200001,200.656158,131.399994,580.951111,209.929993,224.083328,152.899994,91.666664,669.748718,...,85.725021,156.370575,126.616669,2600.000000,229.981812,480.975006,261.250000,287.000000,45.664318,434.789032
2008-01-04,79.237503,43.500000,200.203018,135.000000,563.823486,214.889999,226.000000,153.199997,92.800003,690.275513,...,87.750023,157.922775,129.133331,2604.699951,236.363632,481.250000,255.725006,286.750000,49.356327,432.107788
2008-01-07,79.000000,42.400002,198.481033,134.500000,559.194397,219.399994,223.666672,153.289993,88.333336,692.607056,...,86.175018,154.799194,129.666672,2599.000000,250.899994,474.674988,252.199997,278.750000,49.356327,427.105804


### 3.3 Define Functions

In [171]:
def defineAutoencoder(num_stock, encoding_dim = 5, verbose=0):
    
    """
    Function for fitting an Autoencoder
    """

    # connect all layers
    input = Input(shape=(num_stock,))

    encoded = Dense(encoding_dim, kernel_regularizer=regularizers.l2(0.00001),name ='Encoder_Input')(input)

    decoded = Dense(num_stock, kernel_regularizer=regularizers.l2(0.00001), name ='Decoder_Input')(encoded)
    decoded = Activation("linear", name='Decoder_Activation_function')(decoded)

    # construct and compile AE model
    autoencoder = Model(inputs=input, outputs=decoded)
    adam = optimizers.Adam(learning_rate=0.0005)
    autoencoder.compile(optimizer=adam, loss='mean_squared_error')
    if verbose!= 0:
        autoencoder.summary()

    return autoencoder

In [172]:
def getReconstructionErrorsDF(df_pct_change, reconstructed_data):
    
    """
    Function for calculating the reconstruction Errors
    """
    array = []
    stocks_ranked = []
    num_columns = reconstructed_data.shape[1]
    for i in range(0, num_columns):
        diff = np.linalg.norm((df_pct_change.iloc[:, i] - reconstructed_data[:, i]))  # 2 norm difference
        array.append(float(diff))

    ranking = np.array(array).argsort()
    r = 1
    for stock_index in ranking:
        stocks_ranked.append([ r
                              ,stock_index
                              ,df_pct_change.iloc[:, stock_index].name
                              ,array[stock_index]
                              ])
        r = r + 1

    columns = ['ranking','stock_index', 'stock_name' ,'recreation_error']
    df = pd.DataFrame(stocks_ranked, columns=columns)
    df = df.set_index('stock_name')
    return df

### 3.4 Get the Percentage Change of the Close Prices

In [173]:
col_names = df.columns.to_list()

In [174]:
print(col_names)

['HCLTECH.NS', 'EICHERMOT.NS', 'HINDALCO.NS', 'INDUSINDBK.NS', 'GRASIM.NS', 'AXISBANK.NS', 'ONGC.NS', 'BRITANNIA.NS', 'BPCL.NS', 'RELIANCE.NS', 'GAIL.NS', 'ASIANPAINT.NS', 'ADANIPORTS.NS', 'KOTAKBANK.NS', 'TATASTEEL.NS', 'TITAN.NS', 'M&M.NS', 'SUNPHARMA.NS', 'INFY.NS', 'SHREECEM.NS', 'SBIN.NS', 'HINDUNILVR.NS', 'WIPRO.NS', 'NTPC.NS', 'BAJAJ-AUTO.NS', 'DRREDDY.NS', 'CIPLA.NS', 'NESTLEIND.NS', 'ITC.NS', 'MARUTI.NS', 'HEROMOTOCO.NS', 'LT.NS', 'ULTRACEMCO.NS', 'IOC.NS', 'HDFCBANK.NS', 'JSWSTEEL.NS', 'POWERGRID.NS', 'TATAMOTORS.NS', 'UPL.NS', 'BAJAJFINSV.NS', 'ICICIBANK.NS', 'DIVISLAB.NS', 'TCS.NS', 'TECHM.NS', 'BAJFINANCE.NS', 'BHARTIARTL.NS']


In [175]:
df_pct_change = df.pct_change(1).astype(float)
df_pct_change = df_pct_change.replace([np.inf, -np.inf], np.nan)
df_pct_change = df_pct_change.fillna(method='bfill')

# the percentage change function will make the first two rows equal to nan
df_pct_change = df_pct_change.tail(len(df_pct_change) - 2)

/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/ipykernel_52948/2395198504.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pct_change = df_pct_change.fillna(method='bfill')


In [176]:
if df_pct_change.isnull().values.any():
    print("Warning: NaN values detected in df_pct_change. Please handle them before calculating reconstruction errors.")


In [177]:
df_pct_change.shape

(3982, 46)

In [178]:
# remove columns where there is no change over a longer time period
df_pct_change = df_pct_change[df_pct_change.columns[((df_pct_change == 0).mean() <= 0.05)]]

In [179]:
df_pct_change.head()

,HCLTECH.NS,EICHERMOT.NS,HINDALCO.NS,INDUSINDBK.NS,GRASIM.NS,AXISBANK.NS,ONGC.NS,BRITANNIA.NS,BPCL.NS,RELIANCE.NS,...,POWERGRID.NS,TATAMOTORS.NS,UPL.NS,BAJAJFINSV.NS,ICICIBANK.NS,DIVISLAB.NS,TCS.NS,TECHM.NS,BAJFINANCE.NS,BHARTIARTL.NS
date,,,,,,,,,,,,,,,,,,,,,
2008-01-03,-0.028076,0.126829,0.007738,-0.006051,-0.015686,0.004498,0.043057,0.006186,0.040681,0.015950,...,0.031821,0.023839,0.005027,-0.011031,-0.026775,0.001562,-0.015080,-0.000740,-0.004237,-0.018164
2008-01-04,0.000631,-0.058442,-0.002258,0.027397,-0.029482,0.023627,0.008553,0.001962,0.012364,0.030649,...,0.023622,0.009926,0.019876,0.001808,0.027749,0.000572,-0.021148,-0.000871,0.080851,-0.006167
2008-01-07,-0.002997,-0.025287,-0.008601,-0.003704,-0.008210,0.020987,-0.010324,0.000587,-0.048132,0.003378,...,-0.017949,-0.019779,0.004130,-0.002188,0.061500,-0.013662,-0.013784,-0.027899,0.000000,-0.011576
2008-01-08,-0.030380,-0.009552,-0.002740,-0.003717,-0.017936,0.025524,-0.004471,0.076391,-0.004340,0.017822,...,-0.007833,-0.010275,0.010283,-0.000039,0.007283,-0.001949,-0.016653,0.004484,0.061713,0.034501
2008-01-09,0.006527,-0.036790,-0.017170,-0.046269,-0.019106,-0.005333,-0.004491,-0.052273,-0.014781,-0.004540,...,-0.043092,-0.015635,0.020356,-0.007349,-0.025288,-0.013193,0.014113,0.004464,-0.023269,0.002550


### 3.5 Construct the Autoencoder

In [180]:
# define the input parameters
hidden_layers = 5
batch_size = 500
epochs = 500
stock_selection_number = 20
num_stock = df_pct_change.shape[1]
verbose = 1

In [181]:
print('-' * 20 + 'Step 1 : Returns vs. recreation error (recreation_error)')
print('-' * 25 + 'Transform dataset with MinMax Scaler')

--------------------Step 1 : Returns vs. recreation error (recreation_error)
-------------------------Transform dataset with MinMax Scaler


In [182]:
# Normalize the data
df_scaler = preprocessing.MinMaxScaler()
df_pct_change_normalised = df_scaler.fit_transform(df_pct_change)

In [183]:
# define autoencoder
print('-' * 25 + 'Define autoencoder model')
num_stock = len(df_pct_change.columns)
autoencoder = defineAutoencoder(num_stock=num_stock, encoding_dim=hidden_layers, verbose=verbose)
#plot_model(autoencoder, to_file='img/model_autoencoder_1.png', show_shapes=True,
#           show_layer_names=True)

-------------------------Define autoencoder model


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 45)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Encoder_Input (Dense)           │ (None, 5)              │           230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder_Input (Dense)           │ (None, 45)             │           270 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder_Activation_function     │ (None, 45)             │             0 │
│ (Activation)                    │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 500 (1.95 KB)

 Trainable params: 500 (1.95 KB)

 Non-trainable params: 0 (0.00 B)

In [184]:
# train autoencoder
print('-' * 25 + 'Train autoencoder model')
autoencoder.fit(df_pct_change_normalised, df_pct_change_normalised, shuffle=False, epochs=epochs,
                batch_size=batch_size,
                verbose=verbose)

-------------------------Train autoencoder model
Epoch 1/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - loss: 0.1890
Epoch 2/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - loss: 0.1706
Epoch 3/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - loss: 0.1557
Epoch 4/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 0.1434
Epoch 5/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - loss: 0.1326
Epoch 6/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 0.1225
Epoch 7/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - loss: 0.1131
Epoch 8/500


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - loss: 0.1042
Epoch 9/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0957 
Epoch 10/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0877 
Epoch 11/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - loss: 0.0801
Epoch 12/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - loss: 0.0729
Epoch 13/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - loss: 0.0661
Epoch 14/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - loss: 0.0596
Epoch 15/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - loss: 0.0536
Epoch 16/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 455us/step - loss: 0.0480
Epoch 17/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - loss: 0.0429
Epoch 18/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step - loss: 0.0381
Epoch 19/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - loss: 0.0338
Epoch 20/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - loss: 0.0299
Epoch 21/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - loss: 0.0263
Epoch 22/500
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step - loss: 0.0232
Epoch 23/50

In [185]:
# predict autoencoder
print('-' * 25 + 'Predict autoencoder model')
reconstruct = autoencoder.predict(df_pct_change_normalised)

-------------------------Predict autoencoder model
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step


In [186]:
# Inverse transform dataset with MinMax Scaler
print('-' * 25 + 'Inverse transform dataset with MinMax Scaler')
reconstruct_real = df_scaler.inverse_transform(reconstruct)
df_reconstruct_real = pd.DataFrame(data=reconstruct_real, columns=df_pct_change.columns)

-------------------------Inverse transform dataset with MinMax Scaler


In [187]:
print('-' * 25 + 'Calculate L2 norm as reconstruction loss metric')
df_recreation_error = getReconstructionErrorsDF(df_pct_change=df_pct_change,
                                                reconstructed_data=reconstruct_real)

-------------------------Calculate L2 norm as reconstruction loss metric


In [188]:
df_recreation_error

,ranking,stock_index,recreation_error
stock_name,,,
ITC.NS,1,27,0.849437
NTPC.NS,2,23,0.858446
HDFCBANK.NS,3,33,0.859710
HINDUNILVR.NS,4,21,0.859882
CIPLA.NS,5,26,0.905599
GRASIM.NS,6,4,0.918263
LT.NS,7,30,0.920663
ASIANPAINT.NS,8,11,0.926349
MARUTI.NS,9,28,0.952041


In [189]:
filtered_stocks = df_recreation_error.head(stock_selection_number).index

In [190]:
filtered_stocks

Index(['ITC.NS', 'NTPC.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS', 'CIPLA.NS',
       'GRASIM.NS', 'LT.NS', 'ASIANPAINT.NS', 'MARUTI.NS', 'RELIANCE.NS',
       'POWERGRID.NS', 'SUNPHARMA.NS', 'WIPRO.NS', 'TCS.NS', 'DRREDDY.NS',
       'INFY.NS', 'GAIL.NS', 'SBIN.NS', 'ICICIBANK.NS', 'HEROMOTOCO.NS'],
      dtype='object', name='stock_name')

In [191]:
# store the list of selected stocks
%store filtered_stocks

Stored 'filtered_stocks' (Index)


In [192]:
pip install pickleshare

Note: you may need to restart the kernel to use updated packages.


In [193]:
%store

Stored variables and their in-db values:
df_close_full_stocks             ->             date   HCLTECH.NS  EICHERMOT.NS  HINDA
filtered_stocks                  -> Index(['ITC.NS', 'NTPC.NS', 'HDFCBANK.NS', 'HINDUN
